In [2]:
import osmnx as ox
import networkx as nx
import folium

# Obtener el grafo del distrito de La Punta en Lima, Perú
place_name = "La punta, Lima, Perú"
graph = ox.graph_from_place(place_name, network_type="drive")

# Coordenadas de origen (ejemplo)
orig_coord = (-12.071320, -77.165400)

# Obtener el nodo más cercano a la ubicación de origen
source_node = ox.distance.nearest_nodes(graph, orig_coord[1], orig_coord[0])

# Obtener el subgrafo conexo más grande
largest_subgraph = max(nx.strongly_connected_components(graph), key=len)

# Crear un grafo dirigido que contenga solo los nodos del subgrafo conexo más grande
subgraph = graph.subgraph(largest_subgraph)

# Calcular una ruta aproximada que pase por todos los nodos posibles y regrese al origen
route = list(nx.dfs_preorder_nodes(subgraph, source=source_node))

# Agregar el nodo de origen al final de la ruta para regresar al punto de partida
route.append(route[0])

# Crear un mapa centrado en el lugar
m = folium.Map(location=[orig_coord[0], orig_coord[1]], zoom_start=15, control_scale=True)

# Dibujar los nodos y las aristas del grafo en el mapa
folium.features.GeoJson(ox.graph_to_gdfs(graph, edges=False).to_json()).add_to(m)

# Dibujar la ruta en el mapa
edges = list(zip(route, route[1:]))
for u, v in edges:
    folium.PolyLine(locations=[(graph.nodes[u]['y'], graph.nodes[u]['x']), (graph.nodes[v]['y'], graph.nodes[v]['x'])],
                    color='red').add_to(m)

# Guardar el mapa en un archivo HTML
m.save('map.html')


In [3]:
import csv

# Obtener todos los nodos de la ciudad de La Punta
nodes = list(subgraph.nodes())

# Crear un archivo CSV
with open('coordenadas.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Escribir la cabecera del archivo CSV
    writer.writerow(['latitud', 'longitud'])

    # Escribir las coordenadas de los nodos en el archivo CSV
    for node in nodes:
        lat = graph.nodes[node]['y']
        lon = graph.nodes[node]['x']
        writer.writerow([lat, lon])

print("Archivo CSV creado exitosamente.")


Archivo CSV creado exitosamente.
